In [24]:
import sys,os,re,glob,shutil,pickle,subprocess
import pandas as pd
import numpy as np
import time
%run '~/jupyter/py3_functions.py'
os.chdir('/n/home11/twooldridge/museum_data/')


Split specimen data into a bunch of parts

In [32]:
!mkdir -p smallfiles
infile = 'fulldata_processed_20181221_coordinatesonly.txt'
line_num=1000
cmd='split -l %s %s smallfiles/specimens_input_' % (line_num,infile)
!$cmd

Run Rscript to perform join with raster and daymet data. It needs a decent amount of memory to deal with the large data frames, but doesn't take that long

In [33]:
!mkdir -p cmds
!mkdir -p logs
files = glob.glob('smallfiles/*input*')
for file in files:
    output = re.sub('input','env',file)
    cmd = 'Rscript --vanilla fuzzy_env_join.R %s %s' % (file,output)
    slurm = make_slurm(run=True,id='%s.env2' % re.sub('smallfiles/','',file),p='serial_requeue',cmd_string=cmd,mem='5000',time='00:03:30')

Recombine all the files 

In [34]:
%%bash
cat smallfiles/*env* > fulldata_processed_20181221_env.txt